## Check your connection to Generative AI Hub


👉 First you need to assign the values from `generative-ai-codejam/.aicore-config.json` to the environmental variables. 

That way the Generative AI Hub [Python SDK](https://pypi.org/project/generative-ai-hub-sdk/) will connect to Generative AI Hub.

👉 For the Python SDK to know which resource group to use, you also need to set the `resource group` in the [`variables.py`](variables.py) file to your own `resource group` (e.g. **team-01**) that you created in the SAP AI Launchpad in exercise [00-connect-AICore-and-AILaunchpad](000-connect-AICore-and-AILaunchpad.md).

In [1]:
!pip show generative-ai-hub-sdk

Name: generative-ai-hub-sdk
Version: 3.1.0
Summary: generative AI hub SDK
Home-page: https://www.sap.com/
Author: SAP SE
Author-email: 
License: SAP DEVELOPER LICENSE AGREEMENT
Location: /Users/I066288/opt/anaconda3/lib/python3.9/site-packages
Requires: ai-core-sdk, click, dacite, langchain, langchain-community, openai, overloading, packaging, pydantic
Required-by: 


In [2]:
import json
import os
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
# Inline credentials
with open('creds.json') as f:
    credCF = json.load(f)
 
# Set environment variables
def set_environment_vars(credCF):
    env_vars = {
        'AICORE_AUTH_URL': credCF['url'] + '/oauth/token',
        'AICORE_CLIENT_ID': credCF['clientid'],
        'AICORE_CLIENT_SECRET': credCF['clientsecret'],
        'AICORE_BASE_URL': credCF["serviceurls"]["AI_API_URL"] + "/v2",
        'AICORE_RESOURCE_GROUP': "llm-deployed"
    }
 
    for key, value in env_vars.items():
        os.environ[key] = value
        print(value)
 
# Create AI Core client instance
def create_ai_core_client(credCF):
    set_environment_vars(credCF)  # Ensure environment variables are set
    return AICoreV2Client(
        base_url=os.environ['AICORE_BASE_URL'],
        auth_url=os.environ['AICORE_AUTH_URL'],
        client_id=os.environ['AICORE_CLIENT_ID'],
        client_secret=os.environ['AICORE_CLIENT_SECRET'],
        resource_group=os.environ['AICORE_RESOURCE_GROUP']
    )
 
ai_core_client = create_ai_core_client(credCF)

https://anuragv2-39wjy902.authentication.eu10.hana.ondemand.com/oauth/token
sb-adcd4907-f1a7-462d-ba8e-646390ee4185!b398425|aicore!b540
107d3e9b-9a41-4b30-9b24-a091a45956cd$VmYxjzammFm50xkj1O37HmzgX3maoNrwfrlm99qUhi0=
https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2
llm-deployed


In [3]:
# #import init_env
# #import variables
# #import importlib
# #variables = importlib.reload(variables)

# # TODO: You need to specify which model you want to use. In this case we are directing our prompt
# # to the openAI API directly so you need to pick one of the GPT models. Make sure the model is actually deployed
# # in genAI Hub. You might also want to chose a model that can also process images here already. 
# # E.g. 'gpt-4o-mini' or 'gpt-4o'
# MODEL_NAME = 'gpt-4o'
# # Do not modify the `assert` line below
# assert MODEL_NAME!='', """You should change the variable `MODEL_NAME` with the name of your deployed model (like 'gpt-4o-mini') first!"""

# #init_env.set_environment_variables()
# # Do not modify the `assert` line below 
# assert variables.RESOURCE_GROUP!='', """You should change the value assigned to the `RESOURCE_GROUP` in the `variables.py` file to your own resource group first!"""
# print(f"Resource group is set to: {variables.RESOURCE_GROUP}")

## Prompt an LLM with Generative AI Hub

In [ ]:
from gen_ai_hub.proxy.native.openai import chat
from IPython.display import Markdown

messages = [
    {
        "role": "user", 
        "content": "What is the underlying model architecture of an LLM? Explain it as short as possible."
    }
]

kwargs = dict(model_name="gpt-4o", messages=messages)

response = chat.completions.create(**kwargs)
display(Markdown(response.choices[0].message.content))

## Understanding roles
Most LLMs have the roles `system`, `assistant` (GPT) or `model` (Gemini) and `user` that can be used to steer the models response. In the previous step you only used the role `user` to ask your question. 

👉 Try out different `system` messages to change the response. You can also tell the model to not engage in smalltalk or only answer questions on a certain topic. Then try different user prompts as well!

In [ ]:
messages = [
    {   "role": "system", 
        # TODO try changing the system prompt
        "content": "Speak like Yoda from Star Wars."
    }, 
    {
        "role": "user", 
        "content": "What is the underlying model architecture of an LLM? Explain it as short as possible."
    }
]

kwargs = dict(model_name="gpt-4o", messages=messages)
response = chat.completions.create(**kwargs)
display(Markdown(response.choices[0].message.content))

👉 Also try to have it speak like a pirate.

👉 Now let's be more serious! Tell it to behave like an SAP consultant talking to AI Developers.

👉 Ask it to behave like an SAP Consultant talking to ABAP Developers and to make ABAP comparisons.

## Hallucinations
👉 Run the following question.

In [ ]:
messages = [
    {   "role": "system", 
        "content": "You are an SAP Consultant."
    }, 
    {
        "role": "user", 
        "content": "How does the data masking of the orchestration service work?"
    }
]

kwargs = dict(model_name="gpt-4o", messages=messages)
response = chat.completions.create(**kwargs)
display(Markdown(response.choices[0].message.content))

☝️ Compare the response to [SAP Help - Generative AI Hub SDK](https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/orchestration-service.html). 

👉 What did the model respond? Was it the truth or a hallucination?

👉 Which questions work well, which questions still do not work so well?

# Use Multimodal Models

Multimodal models can use different inputs such as text, audio and images. In Generative AI Hub on SAP AI Core you can access multiple multimodal models (e.g. `gpt-4o-mini`).

👉 If you have not deployed one of the gpt-4o-mini models in previous exercises, then go back to the model library and deploy a model that can also process images.

👉 Now run the code snippet below to get a description for [the image of the AI Foundation Architecture](documents/ai-foundation-architecture.png). These descriptions can then for example be used as alternative text for screen readers or other assistive tech.

👉 You can upload your own image and play around with it!

In [ ]:
import base64

# get the image from the documents folder
with open("documents/ai-foundation-architecture.png", "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

messages = [{"role": "user", "content": [
            {"type": "text", "text": "Describe the images as an alternative text."},
            {"type": "image_url", "image_url": {
                "url": f"data:image/png;base64,{image_data}"}
            }]
        }]

kwargs = dict(model_name="gpt-4o", messages=messages)
response = chat.completions.create(**kwargs)

display(Markdown(response.choices[0].message.content))

# Extracting text from images
Nora loves bananabread and thinks recipes are a good example of how LLMs can also extract complex text from images, like from [a picture of a recipe of a bananabread](documents/bananabread.png). Try your own recipe if you like :)

This exercise also shows how you can use the output of an LLM in other systems, as you can tell the LLM how to output information, for example in JSON format.

In [ ]:
# get the image from the documents folder
with open("documents/bananabread.png", "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

messages = [{"role": "user", "content": [
            {"type": "text", "text": "Extract the ingredients and instructions in two different json files."},
            {"type": "image_url", "image_url": {
                "url": f"data:image/png;base64,{image_data}"}
            }]
        }]

kwargs = dict(model_name="gpt-4o", messages=messages)
response = chat.completions.create(**kwargs)

display(Markdown(response.choices[0].message.content))

[Next exercise](04-create-embeddings.ipynb)
